## Análise utilizando algoritmos de ML

### Pontos importantes
- frequencias estimuladas (total de 40): 8.0, 8.2, ... 15.16, 15.18.
- taxa de amostragem: 250 Hz

### Analisar os 'momentos' em que ocorrem evocação do sinal SSVEP

1. Criar o objeto `MNE` a partir dos dados do participante;
2. Aplicar no objeto  `MNE` o filtro passa-faixa nos valroes de 6 - 18 Hz;
3. Criar cópias do `MNE` com fatias de tempo menores para analisar momentos que ocorrem estimulos ou não;
  - 0.0 - 0.5 segundos e 2.5 - 3.0 segundos ocorre apenas ruído;
  - 0.5 - 2.5 segundos ocorre sinal SSVEP (com ruídos);
4. Com so sinais separados em objetos `MNE`, aplicar a `FFT`, para que seja possível plotar gráficos que contenham (ou não) as informações.
  - Os dados devem ser plotados no dominio da frequencia (após a transformada de Fourier). O FFT pode ser realizado pela biblioteca scipy.fft.
  - Deve ser observado que as janelas (a) com o ruído não aparecerão de fato o sinal SSVEP.

  ### Extração de características

Uma característica importante de acordo com o artigo base do dataset `BETA` é o *signal-to-noise ratio* (SNR).

```Incluir equação SNR```

Ao final desta etapa, será obtido um vetor de características.
Estas podem ser:
- `SNR` (obrigatósia);
- Maior valor espectral (FFT);
- Média dos valores espectrais (FFT)

Dimensionalidade dos dados será da seguinte forma:

`40 , 4 , 64 , 750` -> 40 targets, 4 trials, 64 canais e 750 valores
`160 , 64 (SNR) + 64 (média) + 64  (maior) ...`
Resultando em `160, 192`.

### Seleção de características e classificação

Como existem diversos eletrodos (canais) que nao obtém sinal SSVEP, podemos extrair características que não contribuem para a classificação dos dados.

Podemos utilizar o método `RFE` (*Recursive Feature Elimination*) aplicando por meio de `sklearn.feature_selection.RFE`, aprimorando o parâmetro `n_features_to_select` até obter o melhor resultado de classificação.

Para a classificação propriamente fita, é considerando o uso do método `SVM.`

In [83]:
# imports

from scipy.io import loadmat
import numpy as np
import mne
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_selection import RFE
from sklearn import svm
from sklearn.svm import SVC

In [72]:
# pré-carregamento

beta = loadmat(f"datasets/beta/S2.mat")

data = beta['data'][0][0][0]
freqs = beta['data'][0][0][1]['freqs'][0][0]

data = data.reshape(data.shape[2]*data.shape[3], data.shape[0], data.shape[1])
frequencies = np.array( list(freqs.flatten())*4)


print(data.shape)
print(frequencies.shape)

(160, 64, 750)
(160,)


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
from scipy import signal
from sklearn.preprocessing import LabelEncoder
import mne


# definições de filtros

def butter_bandpass(data, lowcut, highcut, fs=512, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='bandpass')
    return signal.filtfilt(b, a, data)


def butter_lowpass(data, lowcut, fs=512, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    b, a = signal.butter(order, low, btype='lowpass')
    return signal.filtfilt(b, a, data)


def butter_highpass(data, highcut, fs=512, order=4):
    nyq = fs * 0.5
    high = highcut / nyq
    b, a = signal.butter(order, high, btype='highpass')
    return signal.filtfilt(b, a, data)


def butter_notch(data, cutoff, var=1, fs=512, order=4):
    nyq = fs * 0.5
    low = (cutoff - var) / nyq
    high = (cutoff + var) / nyq
    b, a = signal.iirfilter(order, [low, high], btype='bandstop', ftype="butter")
    return signal.filtfilt(b, a, data)

In [4]:

def print_graphs(data):
    for i in range(0, 84, 3):
        plt.plot(data[i,:])
    plt.title('Domínio do tempo')
    plt.show()

    for i in range(0, 84, 3):
        plt.psd(data[i,:], Fs=512)
    plt.title('Domínio da frequência')
    plt.show()

    for i in range(0, 84, 3):
        plt.specgram(data[i,:], Fs=512)
    plt.title('Domínio da frequência')
    plt.show()

In [73]:
# criação do objeto MNE
sfreq = 250
ch_names = [beta['data'][0][0][1]['chan'][0][0][i][3][0] for i in range(64)]
ch_types = ['eeg'] * 64
info = mne.create_info(ch_names, sfreq=sfreq, ch_types=ch_types)
info    


<Info | 7 non-empty values
 bads: []
 ch_names: FP1, FPZ, FP2, AF3, AF4, F7, F5, F3, F1, FZ, F2, F4, F6, F8, ...
 chs: 64 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: unspecified
 nchan: 64
 projs: []
 sfreq: 250.0 Hz
>

In [74]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
events = np.column_stack((
    np.array(range(len(frequencies))),
    np.zeros(len(frequencies), dtype=int),
    le.fit_transform(frequencies))
)
event_dict = {'8.6': 3, '8.8': 4, '9.0': 5, '9.2': 6, '9.4': 7, '9.6': 8, '9.8': 9, '10.0': 10, '10.2': 11, '10.4': 12, '10.6': 13, '10.8': 14, '11.0': 15, '11.2': 16, '11.4': 17, '11.6': 18, '11.8': 19, '12.0': 20, '12.2': 21, '12.4': 22, '12.6': 23, '12.8': 24, '13.0': 25, '13.2': 26, '13.4': 27, '13.6': 28, '13.8': 29, '14.0': 30, '14.2': 31, '14.4': 32, '14.6': 33, '14.8': 34, '15.0': 35, '15.2': 36, '15.4': 37, '15.6': 38, '15.8': 39,  '8.0': 0, '8.2': 1, '8.4': 2}
mne_data = mne.EpochsArray(data, info, events, event_id=event_dict)


Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated


In [77]:
# Estimando o ruído de fundo

# intervalos de tempo sem estímulo (0 a 0,5 segundos e 2,5 a 3 segundos)
base_start = 0
base_end = 125
rest_start = 625
rest_end = 750

# armazena uma lista com as médias de potência para cada canal
noise_power = []
# consideramos a primeira amostra (1º target, 1º trial)
for channel_data in data[0, :, :]:
    fft_result = np.fft.fft(channel_data)
    # densidade espectral de potência (PSD)
    psd = np.abs(fft_result) ** 2
    # média da potência nos intervalos de tempo sem estímulo
    base_power = np.mean(psd[base_start:base_end])
    rest_power = np.mean(psd[rest_start:rest_end])
    # média das duas médias de potência obtidas anteriormente
    mean_noise_power = (base_power + rest_power) / 2
    noise_power.append(mean_noise_power)
# média das médias de potência de todos os canais para estimar o ruído de fundo
estimated_background_noise = np.mean(noise_power)
estimated_background_noise

81464.7871303496

In [78]:
sr = 250

# Frequências alvo
target_frequencies = np.arange(8, 16, 0.2)
# lista para armazenar as amplitudes nas frequências alvo
target_amplitudes = []

for channel_data in mne_data.get_data():
    aux = []
    for eletrodo in channel_data:
        fft_result = np.fft.fft(eletrodo)
        psd = np.abs(fft_result) ** 2
        freqs = np.fft.fftfreq(len(fft_result), 1 / sr)
        target_amplitudes_trial = []
        for target_frequency in target_frequencies:
            # Encontrando o índice da frequência alvo no espectro de frequência
            index = np.argmin(np.abs(freqs - target_frequency))
            # Amplitude na frequência alvo
            amplitude = np.sqrt(psd[index])
            target_amplitudes_trial.append(amplitude)
        aux.append(target_amplitudes_trial)
    
    target_amplitudes.append(aux)

target_amplitudes = np.array(target_amplitudes)
frequencies.shape

(160,)

In [79]:
target_amplitudes_adjusted = np.abs(target_amplitudes - mean_noise_power)

narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / mean_noise_power)

total_power = np.sum(target_amplitudes_adjusted)
wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)

data = np.array([narrow_band_SNR, wide_band_SNR])
data = data.swapaxes(0, 1)
data = data.swapaxes(1, 2)
data = data.reshape(data.shape[0], data.shape[1], data.shape[2] * data.shape[3])
print(data.shape)


(160, 64, 80)


Seleção de características manual utilizando SVM

In [81]:
desired_channels = ['PZ', 'PO3', 'PO5', 'PO4', 'PO6', 'POZ', 'O1', 'OZ', 'O2']
desired_channels = np.isin(ch_names, desired_channels)
filtered_channels = data[:, desired_channels, :]

manualSelection_channels = filtered_channels.reshape(filtered_channels.shape[0], filtered_channels.shape[1] * filtered_channels.shape[2])
print(manualSelection_channels.shape)

X = StandardScaler().fit_transform(manualSelection_channels)
y = LabelEncoder().fit_transform(frequencies)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

svm_model = SVC(kernel='linear', C=1)
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

(160, 720)
Accuracy: 0.0


Seleção de características automática utilizando RFE

In [90]:
rfe_data = data.reshape(data.shape[0], data.shape[1] * data.shape[2])
print(rfe_data.shape)

X = StandardScaler().fit_transform(rfe_data)
y = LabelEncoder().fit_transform(frequencies)

stratified_kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

svm_model = SVC(kernel='linear')
rfe = RFE(estimator=svm_model, n_features_to_select=3)  # Escolha o número desejado de recursos

accuracies = []

for train_index, test_index in stratified_kfold.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    rfe.fit(X_train, y_train)  # Seleciona as melhores características com base no conjunto de treinamento

    # Obtenha o conjunto de características selecionadas
    selected_features = X_train[:, rfe.support_]

    # Treine seu modelo SVM no conjunto de características selecionadas
    svm_model.fit(selected_features, y_train)

    # Aplique a seleção de recursos ao conjunto de teste
    X_test_selected = X_test[:, rfe.support_]

    # Faça previsões e calcule a precisão
    y_pred = svm_model.predict(X_test_selected)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

# Calcule a média das precisões das dobras
mean_accuracy = sum(accuracies) / len(accuracies)


(160, 5120)


In [91]:
mean_accuracy

0.018634987188446306